# Web Scrapping (Without API)

In this case we dont use the APIS of this pages for get their datse. We going to get the maximum, minimum and mean temperature in Madrid for the actual day from AEMET.The previous temperatures of the day respect to the query hour d´ont appear but not concern us for this practice exercise

In [85]:
import requests, json, datetime, pandas as pd
from bs4 import BeautifulSoup, re

In [86]:
#We get the pages with the prediction by hours
aemet = requests.get('http://www.aemet.es/es/eltiempo/prediccion/municipios/horas/madrid-id28079')
#weather = requests.get('https://weather.com/es-US/tiempo/horario/l/SPXX0050:1:SP')

assert (aemet.status_code == 200) , "Error connecting with aemet.es"
#assert (weather.status_code == 200) , "Error connecting with weather.com"

print "Connection stablished"

Connection stablished


In [87]:
#Aemet
soup_aemet = BeautifulSoup(aemet.content,'lxml')
current_hour = datetime.datetime.now().hour
num_hours = 24 - current_hour

''' # We can`t get the temperature by the table because the info it`s painted dinamically by jquery function
hours_colums_total = soup_aemet.find('div', id = 'dia_tabla_0')
#Aemet show 41 hours form actual time. We get the actual date and count the rest hours of the day for get n hours
hours = ('div', class_ = re.compile("^contenedor_popup_"))#[0:num_hours -1]
'''

script_string =soup_aemet.find("script", text=re.compile('datasets')).string

#Now ge got the script that contain our target var. Has blanks and the names hasn`t quotes
datasets = script_string.split('var' )[1].replace('datasets = ',"").strip()[:-1]

#Now we got the value of var. We going get the temps manually since the var not is a dict neither json neither yaml
temp_string = datasets.split("temp")[1].split("data:")[1].replace('}',"").strip()[1:]
temp_clean = "".join(temp_string.split())
temp_string_list = temp_clean.replace(']',']]').split('],')[:num_hours]
temp_string_list = map(lambda x: x.replace('[','').replace(']',''), temp_string_list)

#We have separated list but are string lists, we need numbers
values_temp_list = [int(x.split(',')[1]) for x in temp_string_list]

#Serie with temp as value and hour as index
temp_by_hour = pd.Series(values_temp_list, index=range(current_hour,current_hour + num_hours))
aemet_df = pd.DataFrame({'hour':temp_by_hour.index, 'temp':temp_by_hour.values})
#aemet_df

In [88]:
def get_aemet_max_and_min():
    max_df = aemet_df[aemet_df['temp'] == aemet_df['temp'].max()]
    min_df = aemet_df[aemet_df['temp'] == aemet_df['temp'].min()]
    #max_df.set_value('0', 'temp', 23)
    
    print "AEMET said the maximum temperature will be %d degrees and it will be at the next hours: %s" % (max_df['temp'].iloc[0],",".join(map(str,max_df['hour'].get_values().tolist())))
    print "AEMET said the minimum temperature will be %d degrees and it will be at the next hours: %s" % (min_df['temp'].iloc[0],",".join(map(str,min_df['hour'].get_values().tolist())))


In [89]:
#Top n max and n min
def get_aemet_top_n_temp(n=5):
    #If not specify the n by default is 5. 
    if(n>num_hours):
        n=num_hours
    
    top_max_df = aemet_df.sort_values(by='temp').head(n)
    top_min_df = aemet_df.sort_values(by='temp', ascending=False).head(n)
    
    print "AEMET said the top maximum temperature will be: %s and it will be at the next hours: %s" % (",".join(map(str,top_max_df['hour'].get_values().tolist())),",".join(map(str,top_max_df['hour'].get_values().tolist())))
    print "AEMET said the top minimum temperature will be: %s and it will be at the next hours: %s" % (",".join(map(str,top_min_df['hour'].get_values().tolist())),",".join(map(str,top_min_df['hour'].get_values().tolist())))


In [90]:
get_aemet_max_and_min()
get_aemet_top_n_temp()

AEMET said the maximum temperature will be 23 degrees and it will be at the next hours: 15
AEMET said the minimum temperature will be 14 degrees and it will be at the next hours: 23
AEMET said the top maximum temperature will be: 23,22,21,20,11 and it will be at the next hours: 23,22,21,20,11
AEMET said the top minimum temperature will be: 15,14,16,13,17 and it will be at the next hours: 15,14,16,13,17
